# Predicting Soil Evaporation Rates — Northern Namibia (1959–2022)

**Module:** Artificial Intelligence and Machine Learning (CSMAI21)  
**Dataset:** ERA5 Monthly Reanalysis — Northern Namibia (18–22°S, 15–19°E)  

---

## Objective

Investigate the relationship between:
- **Weather variables**: dewpoint temperature, air temperature, shortwave radiation flux, precipitation rate
- **Soil variables**: soil temperature (0–7 cm), soil moisture (0–7 cm)
- **Target**: monthly evaporation rate (`mer`, kg/m²/s)

**Research question:** Can we accurately predict evaporation rate from co-located soil and weather measurements?

---

## Notebook Structure

1. Setup & Data Loading  
2. Exploratory Data Analysis (EDA)  
3. Data Preprocessing  
4. Machine Learning Models  
5. Deep Learning (TensorFlow)  
6. Results & Conclusions  

## 1. Setup & Data Loading

In [ ]:
import sys
sys.path.insert(0, '..')  # allow imports from src/

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from src.data_loader import load_data, get_features_and_target
from src.preprocessing import split_data, scale_features
from src.models import train_all_models

sns.set_theme(style='whitegrid', palette='muted')
plt.rcParams['figure.dpi'] = 120

DATA_PATH = '../data/raw/north-namibian-monthly-soil.csv'

In [ ]:
df = load_data(DATA_PATH)
print(f'Shape: {df.shape}')
print(f'Date range: {df["date"].min().strftime("%b %Y")} → {df["date"].max().strftime("%b %Y")}')
df.head()

### Variable Reference

| Column | Description | Unit |
|--------|-------------|------|
| `d2m` | 2-metre dewpoint temperature | K |
| `t2m` | 2-metre air temperature | K |
| `mer` | **Evaporation rate (target)** | kg/m²/s |
| `mtdwswrf` | Shortwave radiation flux | W/m² |
| `mtpr` | Mean total precipitation rate | kg/m²/s |
| `stl1` | Soil temperature (0–7 cm) | K |
| `swvl1` | Soil moisture (0–7 cm) | m³/m³ |

In [ ]:
# Summary statistics
df.describe().T.round(6)

In [ ]:
# Data quality check
print('Missing values:', df.isnull().sum().sum())
print('Duplicate rows:', df.duplicated().sum())

## 2. Exploratory Data Analysis

### 2.1 Correlation Heatmap

We use Pearson correlation to identify which variables are most strongly associated with the evaporation rate (`mer`).

In [ ]:
numeric = df.select_dtypes(include='number')
corr = numeric.corr()

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr, vmin=-1, vmax=1, square=True, cmap='coolwarm',
            annot=True, fmt='.2f', linewidths=0.5, ax=ax)
ax.set_title('Namibian Soil Data — Pearson Correlation', fontsize=14)
plt.tight_layout()

**Key finding:** The strongest negative correlations with evaporation rate are:
- Dewpoint (`d2m`): r = **−0.93**
- Soil moisture (`swvl1`): r = **−0.94**
- Precipitation (`mtpr`): r = **−0.88**

Temperature and soil temperature show weaker correlations (r ≈ −0.40).

### 2.2 Variable Distributions

In [ ]:
feature_cols = ['d2m', 't2m', 'mer', 'mtdwswrf', 'mtpr', 'stl1', 'swvl1']
labels = {
    'd2m': 'Dewpoint (K)', 't2m': 'Air Temp (K)', 'mer': 'Evaporation Rate',
    'mtdwswrf': 'Shortwave Radiation (W/m²)', 'mtpr': 'Precipitation Rate',
    'stl1': 'Soil Temp (K)', 'swvl1': 'Soil Moisture (m³/m³)'
}

fig, axes = plt.subplots(3, 3, figsize=(14, 10))
axes = axes.flatten()
skewness = df[feature_cols].skew()

for i, col in enumerate(feature_cols):
    axes[i].hist(df[col], bins=20, edgecolor='white', alpha=0.85, color='#4C72B0')
    axes[i].set_title(f'{labels[col]}\nskew={skewness[col]:.2f}', fontsize=10)

for j in range(len(feature_cols), len(axes)):
    axes[j].set_visible(False)

fig.suptitle('Variable Distributions — Northern Namibia (1959–2022)', fontsize=13)
plt.tight_layout()

**Observations:**
- `mer` (evaporation) is **negatively skewed** (skew = −0.80): most values are close to 0, with a tail of larger negative values
- `mtpr` (precipitation) and `swvl1` (soil moisture) are **right-skewed**: the majority of months are dry, with occasional high-precipitation events
- `d2m`, `t2m`, `stl1` are approximately **symmetric** (skew ≈ 0)

### 2.3 Scatter Plots vs. Evaporation Rate

In [ ]:
predictors = ['d2m', 't2m', 'mtdwswrf', 'mtpr', 'stl1', 'swvl1']

fig, axes = plt.subplots(2, 3, figsize=(14, 8))
axes = axes.flatten()

for i, col in enumerate(predictors):
    r = df[[col, 'mer']].corr().iloc[0, 1]
    axes[i].scatter(df[col], df['mer'], alpha=0.3, s=8, color='#4C72B0')
    axes[i].set_xlabel(labels[col])
    axes[i].set_ylabel('Evaporation Rate')
    axes[i].set_title(f'r = {r:.2f}', fontsize=11)

fig.suptitle('Features vs. Evaporation Rate', fontsize=13)
plt.tight_layout()

## 3. Data Preprocessing

We compare three scalers. For this dataset (approximately normal, few outliers), **StandardScaler** is most appropriate.

In [ ]:
X, y = get_features_and_target(df)
X_train, X_test, y_train, y_test = split_data(X, y, test_size=0.2, random_state=0)

print(f'Train: {X_train.shape}, Test: {X_test.shape}')

# Apply StandardScaler (fit on train only — no leakage)
X_train_s, X_test_s, scaler = scale_features(X_train, X_test, scaler_type='standard')

print(f'\nScaled train mean (should be ~0): {X_train_s.mean(axis=0).round(10)}')
print(f'Scaled train std  (should be ~1): {X_train_s.std(axis=0).round(4)}')

> **Note on leakage prevention:** The scaler is fitted **only** on `X_train`, then `transform()` is applied to both sets. This ensures test data statistics don't influence the scaler.

## 4. Machine Learning Models

We train four regression models:
1. Linear Regression (baseline)
2. Decision Tree (+ 5-fold cross-validation)
3. Random Forest (+ GridSearchCV over `n_estimators`)
4. K-Nearest Neighbours (k=3)

In [ ]:
comparison = train_all_models(
    X, y,
    X_train_s, X_test_s,
    y_train, y_test,
    config={'random_forest': {'n_estimators_grid': [10, 50, 100, 200, 500]}}
)
comparison.style.background_gradient(subset=['r2'], cmap='Greens')

### Why R² ≈ 1?

ERA5 is a **physically consistent reanalysis product** — all variables are derived from the same underlying atmospheric model. The near-deterministic relationships between variables reflect the physics of the system, not data leakage. Cross-validation confirms generalisation.

## 5. Deep Learning (TensorFlow)

A 3-layer MLP as an alternative approach. Key architecture note: the original notebook used `sigmoid` on the output layer, which is incorrect for unbounded regression targets. We use `linear` here.

In [ ]:
from src.deep_learning import train_mlp

result = train_mlp(X, y, epochs=50, use_linear_output=True)
print(f"Final MSE: {result['final_mse']:.4e}")
print(f"Final MAE: {result['final_mae']:.4e}")

## 6. Results & Conclusions

### Summary of Findings

| Finding | Detail |
|---------|--------|
| **Strongest predictors** | Dewpoint (r=−0.93) and soil moisture (r=−0.94) |
| **Weak predictors** | Air temperature and soil temperature (r≈−0.40) |
| **All models R²** | ≈ 1.00 — evaporation rate is near-perfectly predictable from these variables |
| **Best classical model** | Random Forest (MSE ≈ 1.98×10⁻¹⁴, n_estimators=200) |
| **Deep learning** | TF MLP converged to MSE ≈ 3.81×10⁻¹⁰ in 50 epochs |

### Conclusion

Evaporation rate in Northern Namibia is strongly determined by humidity-related variables (dewpoint, soil moisture, precipitation). This aligns with the physical understanding that evaporation depends primarily on the water vapour pressure gradient between the surface and the atmosphere. The near-perfect model performance reflects the physical consistency of ERA5 reanalysis data, where variables are derived from the same underlying numerical weather model.

### Future Work

- Use LSTM/temporal models to exploit seasonal autocorrelation
- Extend spatially across all of Namibia using gridded ERA5 data
- Feature engineering: add lag features, NDVI, land cover type
- Replace sigmoid output activation with linear for the DL model (done here)
- Add prediction intervals via quantile regression forests